## LangChain components

### Install libraries

In [1]:
!pip install -q python-dotenv langchain langchain-openai langchain-community


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: pip install --upgrade pip


### Configure .env and model

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You're a helpful assistant. Keep your answers concise."),
    ("user", "Summarize in 1 sentence: {text}")
])

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

chain = prompt | llm | StrOutputParser()  # LCEL: prompt → model → parser
result = chain.invoke({"text": "LangChain is a library created in 2022, just as the world was beginning to discover the practical possibilities of large language models like GPT-5. Its creators noticed that most LLM-based projects repeated similar patterns – for example, combining prompts into sequences, storing conversation context, querying databases and search engines. Instead of building all this from scratch, they created a library that provides ready-made building blocks for assembling applications. Thanks to LangChain, we can quickly create prototypes and then develop them towards production. The library has become a standard in the world of AI developers because it allows for easy integration of models from different vendors – not only OpenAI, but also Anthropic, HuggingFace, and local open-source models."})
print(result)

LangChain is a 2022 library that streamlines the development of applications using large language models by providing reusable building blocks for common tasks, facilitating rapid prototyping and integration with various AI models.


### Prompt template
LangChain's ChatPromptTemplate is a template for building structured prompts for chat models. It allows you to define roles (system, user, assistant), variables, and message formats to generate LLM prompts in a consistent and reusable manner.

In [4]:
llm = ChatOpenAI(model_name="gpt-4o")
template = """
Specify tags and the main topic of the text.
tags: What are best tags describing text. Give maximum 5 tags separated by comma.
topic: What is the topic of text. Use maximum couple of words

Format response as JSON as below. Return only JSON without markdown tags.
'tags': ['sometag', 'othertag', 'anothertag', 'tag4', 'tag5']
'subject': 'Some subject of text'

text: {input}
"""

prompt_template = ChatPromptTemplate.from_template(template=template)
chain = prompt_template | llm | StrOutputParser()
chain.invoke({"input":"They picked a way among the trees, and their ponies plodded along, carefully avoiding the many writhing and interlacing roots.  There was no undergrowth.  The ground was rising steadily, and as they went forward it seemed that the trees became taller, darker, and thicker. There was no sound, except an occasional drip of moisture falling through the still leaves.  For the moment there was no whispering or movement among the branches; but they all got an uncomfortable feeling that they were being watched with disapproval, deepening to dislike and even enmity.  The feeling steadily grew, until they found themselves looking up quickly, or glancing back over their shoulders, as if they expected a sudden blow."})

"{\n    'tags': ['forest', 'suspense', 'nature', 'mystery', 'adventure'],\n    'subject': 'forest journey'\n}"

### ResponseSchema and OutputParser
LangChain's ResponseSchema defines the expected response format (e.g., JSON fields with name, type, and description) and helps the model return data in a predictable structure.

The OutputParser interprets the raw LLM response and transforms it into the desired form (e.g., Pydantic object, JSON, list), facilitating further use within the application.

In [9]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field
from typing import List

class TextAnalysis(BaseModel):
    tags: List[str] = Field(description="What are best tags describing text. Give maximum 5 tags separated by comma.")
    topic: str = Field(description="What is the topic of text. Use maximum couple of words.")

parser = JsonOutputParser(pydantic_object=TextAnalysis)
instructions = parser.get_format_instructions()

template = """
Specify tags and the main topic of the text. Return only JSON without markdown tags.
tags: What are best tags describing text. Give maximum 5 tags separated by comma.
topic: What is the topic of text. Use maximum couple of words

Format response as JSON as below:
'tags': ['sometag', 'othertag', 'anothertag', 'tag4', 'tag5']
'subject': 'Some subject of text'

text: {input}

{instructions}
"""
prompt = ChatPromptTemplate.from_template(template=template)

llm = ChatOpenAI(temperature=0.0)
chain = prompt | llm | parser
output_dict = chain.invoke({
    "input":"They picked a way among the trees, and their ponies plodded along, carefully avoiding the many writhing and interlacing roots.  There was no undergrowth.  The ground was rising steadily, and as they went forward it seemed that the trees became taller, darker, and thicker. There was no sound, except an occasional drip of moisture falling through the still leaves.  For the moment there was no whispering or movement among the branches; but they all got an uncomfortable feeling that they were being watched with disapproval, deepening to dislike and even enmity.  The feeling steadily grew, until they found themselves looking up quickly, or glancing back over their shoulders, as if they expected a sudden blow.",
    "instructions":instructions})
print(output_dict)


{'tags': ['forest', 'mystery', 'nature', 'ominous', 'adventure'], 'topic': 'Exploring a mysterious forest'}
